# Campaigns report

### Imports

In [1]:
import sys
import psycopg2
import pandas as pd

### Initial setup

In [2]:
conn = psycopg2.connect(host='localhost',
                        port='5432',
                        user='mffais',
                        password='pass',
                        database='bigquery')

### Calculate installs

In [3]:
cursor = conn.cursor()
query = '''
    SELECT campaign,
           COUNT(*) AS install
    FROM (
      SELECT user_pseudo_id,
             traffic_source ->> 'name' AS campaign
      FROM events
      GROUP BY user_pseudo_id, traffic_source ->> 'name'
    ) AS installs_table
    GROUP BY campaign
    ORDER BY campaign
'''
cursor.execute(query)
install = pd.read_sql(query, con=conn)
cursor.close()
install.rename(columns={ 'campaign':'Campaign', 'install':'Install' }, inplace=True)
install

,Campaign,Install
0,(direct),830
1,Google Unnamed Campaign 7625447428,1
2,kevin-auto-savings 50c,412
3,kevin-auto-savings-micro 150c,33
4,kevin-auto-savings-micro 150c us,24
5,kevin-auto-savings-micro 50c,470
6,kevin-auto-savings-micro 50c us,4
7,kevin-minimalist 50c,457
8,kevin-privacy-50c,533
9,New App Promotion,3867


### Calculate uninstalls

In [4]:
cursor = conn.cursor()
query = '''
    SELECT traffic_source ->> 'name' AS campaign,
           COUNT(*) AS uninstall
    FROM events
    WHERE event_name='app_remove'
    GROUP BY campaign
'''
cursor.execute(query)
uninstall = pd.read_sql(query, con=conn)
cursor.close()
uninstall.rename(columns={ 'campaign':'Campaign', 'uninstall':'Uninstall' }, inplace=True)
uninstall

,Campaign,Uninstall
0,(direct),515
1,Google Unnamed Campaign 7625447428,1
2,kevin-auto-savings 50c,260
3,kevin-auto-savings-micro 150c,17
4,kevin-auto-savings-micro 150c us,6
5,kevin-auto-savings-micro 50c,298
6,kevin-auto-savings-micro 50c us,3
7,kevin-minimalist 50c,322
8,kevin-privacy-50c,339
9,New App Promotion,2710


### Calculate usage

In [5]:
cursor = conn.cursor()
query = '''
    SELECT campaign AS campaign,
           AVG(days_used) AS usage
    FROM (
      SELECT user_pseudo_id,
             campaign,
             days_used
      FROM (
        SELECT user_pseudo_id,
               COUNT(*) AS days_used
        FROM (
          SELECT event_date,
                 user_pseudo_id
          FROM events
          GROUP BY event_date, user_pseudo_id
        ) AS users_by_day
        GROUP BY user_pseudo_id
      ) AS days_used
      INNER JOIN (
        SELECT user_pseudo_id AS user_pseudo_idc,
               traffic_source ->> 'name' AS campaign
        FROM events
        GROUP BY user_pseudo_idc, campaign
      ) AS user_campaign
      ON days_used.user_pseudo_id = user_campaign.user_pseudo_idc
      ORDER BY days_used DESC
    ) AS days_used_report
    GROUP BY campaign
    ORDER BY campaign
'''
cursor.execute(query)
usage = pd.read_sql(query, con=conn)
usage.rename(columns={ 'campaign':'Campaign', 'usage':'Usage' }, inplace=True)
usage['Usage'] = usage['Usage'].apply(lambda value: '%.2f' % value)
usage

,Campaign,Usage
0,(direct),1.47
1,Google Unnamed Campaign 7625447428,1.00
2,kevin-auto-savings 50c,1.07
3,kevin-auto-savings-micro 150c,1.00
4,kevin-auto-savings-micro 150c us,1.00
5,kevin-auto-savings-micro 50c,1.14
6,kevin-auto-savings-micro 50c us,1.00
7,kevin-minimalist 50c,1.09
8,kevin-privacy-50c,1.11
9,New App Promotion,1.43


### Merge install, uninstall and average to result table

In [6]:
result = pd.merge(install, uninstall, on='Campaign')
result['Net new install'] = result['Install'] - result['Uninstall']
result = pd.merge(result, usage, on='Campaign')
result

,Campaign,Install,Uninstall,Net new install,Usage
0,(direct),830,515,315,1.47
1,Google Unnamed Campaign 7625447428,1,1,0,1.00
2,kevin-auto-savings 50c,412,260,152,1.07
3,kevin-auto-savings-micro 150c,33,17,16,1.00
4,kevin-auto-savings-micro 150c us,24,6,18,1.00
5,kevin-auto-savings-micro 50c,470,298,172,1.14
6,kevin-auto-savings-micro 50c us,4,3,1,1.00
7,kevin-minimalist 50c,457,322,135,1.09
8,kevin-privacy-50c,533,339,194,1.11
9,New App Promotion,3867,2710,1157,1.43


### Output HTTP Header

In [7]:
print('Content-type: text/csv')
print('Content-Disposition: attachment; filename="campaigns.csv"')
print()

Content-type: text/csv
Content-Disposition: attachment; filename="campaigns.csv"



### Output variables

In [8]:
print('# Title: Campaign report')

# Title: Campaign report


### Ouput result

In [9]:
result.to_csv(sys.stdout, index=False)

Campaign,Install,Uninstall,Net new install,Usage
(direct),830,515,315,1.47
Google Unnamed Campaign 7625447428,1,1,0,1.00
kevin-auto-savings 50c,412,260,152,1.07
kevin-auto-savings-micro 150c,33,17,16,1.00
kevin-auto-savings-micro 150c us,24,6,18,1.00
kevin-auto-savings-micro 50c,470,298,172,1.14
kevin-auto-savings-micro 50c us,4,3,1,1.00
kevin-minimalist 50c,457,322,135,1.09
kevin-privacy-50c,533,339,194,1.11
New App Promotion,3867,2710,1157,1.43
Week1-Search,1,1,0,1.00
,188,96,92,1.37


### Release resources

In [10]:
conn.close()